In [20]:
import pandas as pd
import re

In [21]:
landsat5 = pd.read_csv('./LANDSAT_TM_C1_281139.csv', encoding = "ISO-8859-1", parse_dates=['Acquisition Date'])
landsat5["Acquisition Date"] = pd.to_datetime(landsat5['Acquisition Date'].dt.strftime("%Y%m%d"))

In [22]:
landsat5.shape

(583, 66)

In [29]:
landsat5['Acquisition Date'].dt.year.unique()

array([2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992,
       1991, 1990, 1989, 1988, 1987, 1986, 1985, 1984])

In [7]:
landsat5.sort_values(by='Acquisition Date', inplace=True)
landsat5.reset_index(inplace=True)
landsat5.rename(mapper={"Acquisition Date": "date"}, axis=1, inplace=True)

In [8]:
landsat5_gs = pd.DataFrame(columns=['area', 'date', 'b', 'gs_location'])

In [12]:
land5_lst = []
with open('landsat5.txt', 'r') as file:
    for line in file:
        rex = re.compile(r'.TIF')
        if rex.search(line) != None:
            land5_lst.append(line)
land5_lst[:5]

['gs://gcp-public-data-landsat/LT05/01/165/043/LT05_L1GS_165043_19891226_20180615_01_T2/LT05_L1GS_165043_19891226_20180615_01_T2_B1.TIF\n',
 'gs://gcp-public-data-landsat/LT05/01/165/043/LT05_L1GS_165043_19891226_20180615_01_T2/LT05_L1GS_165043_19891226_20180615_01_T2_B2.TIF\n',
 'gs://gcp-public-data-landsat/LT05/01/165/043/LT05_L1GS_165043_19891226_20180615_01_T2/LT05_L1GS_165043_19891226_20180615_01_T2_B3.TIF\n',
 'gs://gcp-public-data-landsat/LT05/01/165/043/LT05_L1GS_165043_19891226_20180615_01_T2/LT05_L1GS_165043_19891226_20180615_01_T2_B4.TIF\n',
 'gs://gcp-public-data-landsat/LT05/01/165/043/LT05_L1GS_165043_19891226_20180615_01_T2/LT05_L1GS_165043_19891226_20180615_01_T2_B5.TIF\n']

In [13]:
for idx, line in enumerate(land5_lst):
    dic = {}
    spl = line.split("_")
    dic["area"] = spl[2][:3]
    dic["date"] = spl[3]
    dic["b"] = spl[-1].split(".")[0]
    dic["gs_location"] = line
    landsat5_gs.loc[idx] = dic

In [14]:
landsat5_gs.shape

(4920, 4)

In [15]:
landsat5_gs = landsat5_gs[(landsat5_gs.b == 'B4') | (landsat5_gs.b == 'B3')]
landsat5_gs.reset_index(inplace=True)
landsat5_gs["date"] = pd.to_datetime(landsat5_gs["date"])

In [16]:
landsat5_merged = landsat5_gs.merge(landsat5, on="date")

In [18]:
landsat5_merged.isnull().sum().sum()

1524

In [19]:
landsat5_merged.gs_location = landsat5_merged.gs_location.str.replace("\n", "")

In [ ]:
with open('landsat5_link.txt', 'w') as f:
    for item in landsat5_merged.gs_location:
        f.write("%s\n" % item)